In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os.path
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import numpy as np
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [2]:
if os.path.isfile('../data/master_data.csv') and os.stat('../data/master_data.csv').st_size > 0:
    master_df = pd.read_csv('../data/master_data.csv')
else: 
    master_df = pd.DataFrame()


In [3]:
# Setup the transport and client
_transport = RequestsHTTPTransport(
    url='http://209.38.172.107/graphql/',
    use_json=True,
)

client = Client(
    transport = _transport,
    fetch_schema_from_transport=True,
)

# Define Query
query = gql('''
query TopPlayer {
  playerTotalsAll(id: 15781) {
    playerName
    position
    team
    season
    games
    assists
    blocks
    ft
    threeFg
    twoFg
    effectFgPercent
    totalRb
    steals
    points
    id
  }
  playerAdvancedAll(id: 15781) {
      playerName
    position
    team
    season
    minutesPlayed
    games
    usagePercent
    per
    tsPercent
    box
    winShares
    id
  }
}
''')

# Execute and store response

response = client.execute(query)

print(response)

{'playerTotalsAll': [{'playerName': 'Karl Malone*', 'position': 'PF', 'team': 'UTA', 'season': 1997, 'games': 82, 'assists': 368, 'blocks': 48, 'ft': 521, 'threeFg': 0, 'twoFg': 864, 'effectFgPercent': '0.550', 'totalRb': 809, 'steals': 113, 'points': 2249, 'id': '15781'}], 'playerAdvancedAll': [{'playerName': 'Karl Malone*', 'position': 'PF', 'team': 'UTA', 'season': 1997, 'minutesPlayed': 2998, 'games': 82, 'usagePercent': '32.700', 'per': '28.9', 'tsPercent': '0.600', 'box': '8.3', 'winShares': '16.7', 'id': '15781'}]}


In [4]:
df_totals = pd.json_normalize(response['playerTotalsAll'])
df_advanced = pd.json_normalize(response['playerAdvancedAll'])

df_combined = pd.merge(df_totals, df_advanced, on=['playerName', 'position', 'team', 'season', 'games', 'id'])

df_combined.head()

,playerName,position,team,season,games,assists,blocks,ft,threeFg,twoFg,...,totalRb,steals,points,id,minutesPlayed,usagePercent,per,tsPercent,box,winShares
0,Karl Malone*,PF,UTA,1997,82,368,48,521,0,864,...,809,113,2249,15781,2998,32.700,28.9,0.600,8.3,16.7


In [5]:
master_df = pd.concat([master_df, df_combined], ignore_index=True)

# master_df.drop_duplicates(subset='id', keep='last', inplace=True)

master_df.to_csv('../data/master_data.csv', index=False)